# Diplomatura en ciencia de datos, aprendizaje automático y sus aplicaciones - Edición 2023 - FAMAF (UNC)

## Mentoría 16 - ¿Cómo identificar fuga de ventas? Inteligencia artificial aplicada al sector comercial.

### Explorando Patrones de Datos a través de Clustering (TP3)

#### Parte 4: Evaluación de resultados.

**Integrantes:**
- Canalis, Patricio.
- Chevallier-Boutell, Ignacio José.
- Villarroel Torrez, Daniel.

**Mentores:**
- Gonzalez, Lucía
- Lahoz, Nahuel

---
## Librerías y dataset

In [ ]:
import pandas as pd
import seaborn as sns

# Preferencias
pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows',150)
sns.set_context('talk')
sns.set_theme(style='white')

Los datasets que necesitamos son:

In [2]:
# sr10_knn_pca = pd.read_csv('../data/interim/tp3_sr10_knn5_pca12.csv')

---
# Clusterización con k-means

Clusterizamos con K-means usando $K=5$ y luego grafiamos usando un PCA de 2 componentes.